In [1]:
from google.colab import files
from tensorflow.keras import applications

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenet import MobileNet
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

In [2]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/MyDrive/CVProject/'

/content/drive/MyDrive/CVProject


In [4]:
!ls

CVProject.ipynb  CVProject_MobileNet.h5  Dataset  Landmark_Detection_MER.pdf


## **Creating Models**

In [16]:
def get_data(preprocessing_function):
    image_gen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=(0.2, 1),
                                   shear_range=45,
                                   zoom_range=(0.5, 1.5),
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   preprocessing_function=preprocessing_function,
                                   validation_split=0.3)

    train_data = '/content/drive/MyDrive/CVProject/Dataset'

    train_gen = image_gen.flow_from_directory(train_data,
                                              target_size=(224, 224),
                                              class_mode='categorical',
                                              color_mode='rgb',
                                              shuffle=True,
                                              # batch_size=32,
                                              seed=0,
                                              subset='training')

    val_gen = image_gen.flow_from_directory(train_data,
                                            target_size=(224, 224),
                                            class_mode='categorical',
                                            color_mode='rgb',
                                            shuffle=True,
                                            # batch_size=32,
                                            seed=0,
                                            subset='validation')

    return train_gen, val_gen

In [17]:
def get_model(model):
    kwargs =    {'include_top':False,
                 'weights':'imagenet',
                 'input_shape':(224, 224, 3),
                 'pooling':'avg'}
    
    base_model = model(**kwargs)
    
    end_model = models.Sequential()
    
    end_model.add(layers.Dense(128))
    end_model.add(layers.ReLU())
    end_model.add(layers.Dense(128))
    end_model.add(layers.ReLU())
    end_model.add(layers.Dense(18, activation='softmax'))

    model = models.Model(inputs=base_model.input, outputs=end_model(base_model.output))

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## **Training Models**

In [18]:
models_dict = {
    'Xception': {'model': applications.xception.Xception,
                 'preprocess_func': applications.xception.preprocess_input},
    'VGG19': {'model': applications.vgg19.VGG19,
              'preprocess_func': applications.vgg19.preprocess_input},
    'ResNet50V2': {'model': applications.resnet_v2.ResNet50V2,
                   'preprocess_func': applications.resnet_v2.preprocess_input},
    'ResNet101V2': {'model': applications.resnet_v2.ResNet101V2,
                    'preprocess_func': applications.resnet_v2.preprocess_input},
    'InceptionV3': {'model': applications.inception_v3.InceptionV3,
                    'preprocess_func': applications.inception_v3.preprocess_input},
    'MobileNet': {'model': applications.mobilenet.MobileNet,
                  'preprocess_func': applications.mobilenet.preprocess_input},
    'MobileNetV2': {'model': applications.mobilenet_v2.MobileNetV2,
                    'preprocess_func': applications.mobilenet_v2.preprocess_input},
    'DenseNet201': {'model': applications.densenet.DenseNet201,
                    'preprocess_func': applications.densenet.preprocess_input},
    'EfficientNetB7': {'model': applications.efficientnet.EfficientNetB7,
                       'preprocess_func': applications.efficientnet.preprocess_input}
}

In [19]:
for name, model in models_dict.items():
    print(name)

    train_gen, val_gen = get_data(model['preprocess_func'])
    m = get_model(model['model'])

    start = time.perf_counter()

    history = m.fit(train_gen,
                    epochs=5,
                    validation_data = val_gen)

    duration = time.perf_counter() - start

    models_dict[name]['time'] = duration
    models_dict[name]['val_acc'] = history.history['val_accuracy'][-1]
    
    print()

Xception
Found 269 images belonging to 18 classes.
Found 106 images belonging to 18 classes.
Epoch 1/5
9/9 [==============================] - 70s 8s/step - loss: 2.7629 - accuracy: 0.1822 - val_loss: 2.5697 - val_accuracy: 0.2736
Epoch 2/5
9/9 [==============================] - 61s 7s/step - loss: 2.2998 - accuracy: 0.4126 - val_loss: 2.1217 - val_accuracy: 0.4151
Epoch 3/5
9/9 [==============================] - 62s 7s/step - loss: 1.7966 - accuracy: 0.5242 - val_loss: 1.8197 - val_accuracy: 0.4906
Epoch 4/5
9/9 [==============================] - 61s 7s/step - loss: 1.3796 - accuracy: 0.7175 - val_loss: 1.5028 - val_accuracy: 0.6226
Epoch 5/5
9/9 [==============================] - 61s 7s/step - loss: 1.1100 - accuracy: 0.7249 - val_loss: 1.3909 - val_accuracy: 0.5943

VGG19
Found 269 images belonging to 18 classes.
Found 106 images belonging to 18 classes.
Epoch 1/5
9/9 [==============================] - 68s 8s/step - loss: 3.1289 - accuracy: 0.1227 - val_loss: 2.9155 - val_accuracy: 0

## **Evaluating and Visualizing results**

In [20]:
model_results = []

for name, _ in models_dict.items():
    model_results.append([name,
                          models_dict[name]['val_acc'],
                          models_dict[name]['time']])
    
results = pd.DataFrame(model_results,
                       columns = ['Model', 'Validation Accuracy', 'Training Time (sec.)'])
results = results.sort_values(by='Validation Accuracy', ascending=False).reset_index(drop=True)

results

,Model,Validation Accuracy,Training Time (sec.)
0,EfficientNetB7,0.783019,348.738794
1,ResNet101V2,0.707547,367.336831
2,DenseNet201,0.669811,329.872682
3,ResNet50V2,0.632075,327.746289
4,MobileNet,0.622642,314.625982
5,Xception,0.594340,320.818169
6,MobileNetV2,0.556604,314.526212
7,InceptionV3,0.528302,334.037858
8,VGG19,0.462264,320.328263
